In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import seaborn as sns
#from statsmodels.graphics.mosaicplot import mosaic
import io
import requests

%matplotlib inline


#import xgboost as xgb
#from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import tree

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV


from sklearn.metrics import log_loss

### Lectura de datos del CSV y seteo del tipo para cada columna

In [2]:
url_train="https://raw.githubusercontent.com/joacols/TP_orga_de_datos/main/Train_Preprocesado_TP2_Datos_2020-2C-encodeado.csv"
s_train=requests.get(url_train).content
df_train=pd.read_csv(io.StringIO(s_train.decode('utf-8')),

#df = pd.read_csv('Entrenamieto_ECI_2020.csv',
dtype={
'ID': np.int64,
'Pricing, Delivery_Terms_Quote_Approval': np.int64,
'Pricing, Delivery_Terms_Approved': np.int64,
'Bureaucratic_Code_0_Approval': np.int64,
'Bureaucratic_Code_0_Approved': np.int64,
'Submitted_for_Approval': np.int64,
'Opportunity_ID': np.int64,
#'Sales_Contract_No': np.int64,
'Delivery_Year': np.int64,
'Total_Power': np.int64,
#'Price':np.float64,
'ASP':np.float64,
'ASP_(converted)':np.float64,
'Total_Amount':np.float64},
#parse_dates=[
#'Account_Created_Date',
#'Planned_Delivery_Start_Date',
#'Planned_Delivery_End_Date',
#'Opportunity_Created_Date',
#'Last_Activity',
#'Quote_Expiry_Date',
#'Actual_Delivery_Date',
#'Last_Modified_Date'
#]
)

df_train.head()

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Account_Created_Date_Day,Account_Created_Date_Month,Account_Created_Date_Year,Opportunity_ID,Sales_Contract_No,...,Bit_Total_Taxable_Amount_Currency_0,Bit_Total_Taxable_Amount_Currency_1,Bit_Total_Taxable_Amount_Currency_2,Bit_Total_Taxable_Amount_Currency_3,Bit_Prod_Category_A_0,Bit_ASP_Currency_0,Bit_ASP_Currency_1,Bit_ASP_Currency_2,Bit_ASP_Currency_3,Bit_ASP_(converted)_Currency_0
0,0,0,0,0,0,1,3,2017,960,0,...,0,0,0,1,1,0,0,0,1,1
1,0,0,0,0,0,1,3,2017,959,0,...,0,0,0,1,1,0,0,0,1,1
2,0,0,0,0,0,1,3,2017,961,0,...,0,0,0,1,1,0,0,0,1,1
3,0,0,0,0,0,1,3,2017,956,0,...,0,0,0,1,1,0,0,0,1,1
4,0,0,0,0,0,19,9,2014,6985,3681,...,0,0,0,1,1,0,0,0,1,1


In [3]:
url_test="https://raw.githubusercontent.com/joacols/TP_orga_de_datos/main/Test_Preprocesado_TP2_Datos_2020-2C-encodeado.csv"
s_test=requests.get(url_test).content
df_test=pd.read_csv(io.StringIO(s_test.decode('utf-8')),

#df = pd.read_csv('Entrenamieto_ECI_2020.csv',
dtype={
'ID': np.int64,
'Pricing, Delivery_Terms_Quote_Approval': np.int64,
'Pricing, Delivery_Terms_Approved': np.int64,
'Bureaucratic_Code_0_Approval': np.int64,
'Bureaucratic_Code_0_Approved': np.int64,
'Submitted_for_Approval': np.int64,
'Opportunity_ID': np.int64,
#'Sales_Contract_No': np.int64,
'Delivery_Year': np.int64,
'Total_Power': np.int64,
#'Price':np.float64,
'ASP':np.float64,
'ASP_(converted)':np.float64,
'Total_Amount':np.float64},
#parse_dates=[
#'Account_Created_Date',
#'Planned_Delivery_Start_Date',
#'Planned_Delivery_End_Date',
#'Opportunity_Created_Date',
#'Last_Activity',
#'Quote_Expiry_Date',
#'Actual_Delivery_Date',
#'Last_Modified_Date'
#]
)

df_test.head()

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Account_Created_Date_Day,Account_Created_Date_Month,Account_Created_Date_Year,Opportunity_ID,Sales_Contract_No,...,Bit_Total_Taxable_Amount_Currency_0,Bit_Total_Taxable_Amount_Currency_1,Bit_Total_Taxable_Amount_Currency_2,Bit_Total_Taxable_Amount_Currency_3,Bit_Prod_Category_A_0,Bit_ASP_Currency_0,Bit_ASP_Currency_1,Bit_ASP_Currency_2,Bit_ASP_Currency_3,Bit_ASP_(converted)_Currency_0
0,0,0,0,0,0,20,2,2017,12173,0,...,0,0,0,1,1,0,0,0,1,1
1,0,0,0,0,0,22,8,2018,12172,0,...,0,0,0,1,1,0,0,0,1,1
2,0,0,0,0,0,22,8,2018,11408,0,...,0,0,0,1,1,0,0,0,1,1
3,0,0,0,0,0,4,4,2019,12232,0,...,0,0,0,1,1,0,0,0,1,1
4,0,0,0,0,0,4,4,2019,12233,0,...,0,0,0,1,1,0,0,0,1,1


In [4]:
df_train.shape

(9841, 315)

In [5]:
df_test.shape

(1567, 315)

In [6]:
faltantes=pd.DataFrame(df_train.isnull().sum().sort_values(ascending=False),columns=['nulls'])
filas=len(df_train)
faltantes['porcentaje']=round(100*faltantes['nulls']/filas,2)
faltantes[faltantes['nulls']!=0]

,nulls,porcentaje


In [7]:
faltantes=pd.DataFrame(df_test.isnull().sum().sort_values(ascending=False),columns=['nulls'])
filas=len(df_test)
faltantes['porcentaje']=round(100*faltantes['nulls']/filas,2)
faltantes[faltantes['nulls']!=0]

,nulls,porcentaje


# Proceso de preparación de los datos

### Separamos la variable a predecir

In [8]:
#df4_train['Is_closed_won']=(df4_train['Stage']=='Closed Won')
df_train['Is_closed_won']=df_train['Stage'].apply(lambda x: 1 if x=='Closed Won' else 0)
df_train=df_train.drop(['Stage'],axis=1)

In [9]:
X = df_train.drop(['Is_closed_won','Sales_Contract_No'], axis=1)


In [10]:
y = df_train['Is_closed_won']

In [11]:
#X.head()

In [12]:
#y.value_counts()#.head()

### Creamos set de entrenamiento y test

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [14]:
X_train.shape

(7872, 313)

# Proceso de selección de hiperparámetros y predicción usando Decision Tree

## Selección de hiperparámetros, entrenamiento y validación usando Decision Tree

In [36]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import tree
parametros = {
    'class_weight': ["balanced",None],
    'min_samples_split':[2,3,4],
    'min_samples_leaf':[1,2,3],
    'max_depth': [5,10,15,None]
}
decisionTree = tree.DecisionTreeClassifier()
#seleccion de hiperparametros
clf = GridSearchCV(decisionTree, parametros,scoring='neg_log_loss')
clf.fit(X_train, y_train)

C:\Users\licha\Miniconda3\envs\orgadatos\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'class_weight': ['balanced', None], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2, 3], 'max_depth': [5, 10, 15, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [37]:
y_validation =clf.predict(X_test)
y_validation_probs = clf.predict_proba(X_test)[:,1]

In [38]:
print( "Resultado:")
print( " accurancy:" , accuracy_score(y_test, y_validation) )
print( " log loss:" , log_loss(y_test, y_validation_probs) )

Resultado:
 accurancy: 0.7775520568816658
 log loss: 0.5259030600831148


In [39]:
clf.best_params_

{'class_weight': None,
 'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

## Prediccion usando Decision Tree

In [40]:
y_pred_probs =clf.predict_proba( df_test.drop(['Sales_Contract_No','Stage'], axis=1)  )[:,1]

In [41]:
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
prediccion = df_test['Opportunity_ID'].to_frame()
prediccion.insert(1,'Target', y_pred_probs , True) 
#Exportamos la prediccion en formato csv sin labels
prediccion.to_csv(r'./DecisionTree_max_depth_5_class_weight_none_min_samples_leaf_1_min_samples_split_3.csv',index=False , header=True)
prediccion.head()

,Opportunity_ID,Target
0,12173,0.000000
1,12172,0.363636
2,11408,0.714286
3,12232,0.819608
4,12233,0.819608


# Proceso de selección de hiperparámetros y predicción usando Random Forest

## Selección de hiperparámetros,entrenamiento y validación usando Random Forest

In [42]:
parametros = {
    'class_weight': ["balanced",None],
    'min_samples_split':[2,3,4],
    'min_samples_leaf':[1,2,3],
    'max_depth': [5,10,15,None],
    'n_estimators': [50,60,75,100]
}
randomForest = RandomForestClassifier()
#seleccion de hiperparametros
clf = GridSearchCV(randomForest, parametros,scoring='neg_log_loss')
clf.fit(X_train, y_train)

C:\Users\licha\Miniconda3\envs\orgadatos\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'class_weight': ['balanced', None], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2, 3], 'max_depth': [5, 10, 15, None], 'n_estimators': [50, 60, 75, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [43]:
y_validation =clf.predict(X_test)
y_validation_probs = clf.predict_proba(X_test)[:,1]

In [44]:
print( "Resultado:")
print( " accurancy:" , accuracy_score(y_test, y_validation) )
print( " log loss:" , log_loss(y_test, y_validation_probs) )

Resultado:
 accurancy: 0.8466226510919248
 log loss: 0.3724183793669345


In [45]:
clf.best_params_

{'class_weight': None,
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

## Prediccion usando Random Forest

In [46]:
y_pred_probs =clf.predict_proba( df_test.drop(['Sales_Contract_No','Stage'], axis=1)  )[:,1]

In [47]:
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
prediccion = df_test['Opportunity_ID'].to_frame()
prediccion.insert(1,'Target', y_pred_probs , True) 
#Exportamos la prediccion en formato csv sin labels
prediccion.to_csv(r'./RandomForest_max_depth_none_class_weight_balanced_min_samples_leaf_2_min_samples_split_2_n_estimators_100.csv',index=False , header=True)
prediccion.head()

,Opportunity_ID,Target
0,12173,0.52
1,12172,0.46
2,11408,0.68
3,12232,0.60
4,12233,0.61


# Proceso de selección de hiperparámetros y predicción usando Random Forest

## Selección de hiperparámetros,entrenamiento y validación usando Random Forest

In [48]:
parametros = {
    'class_weight': ["balanced",None],
    'min_samples_split':[2,3,4],
    'min_samples_leaf':[1,2,3],
    'max_depth': [5,10,15,None],
    'n_estimators': [50,60,75,100],
    #'max_features':['log2',None]
    #'max_features':['sqrt', 'log2',None]
}
randomForest = RandomForestClassifier(max_features="log2")
#seleccion de hiperparametros
clf = GridSearchCV(randomForest, parametros,scoring='neg_log_loss')
clf.fit(X_train, y_train)

C:\Users\licha\Miniconda3\envs\orgadatos\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'class_weight': ['balanced', None], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2, 3], 'max_depth': [5, 10, 15, None], 'n_estimators': [50, 60, 75, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [49]:
y_validation =clf.predict(X_test)
y_validation_probs = clf.predict_proba(X_test)[:,1]

In [50]:
print( "Resultado:")
print( " accurancy:" , accuracy_score(y_test, y_validation) )
print( " log loss:" , log_loss(y_test, y_validation_probs) )

Resultado:
 accurancy: 0.8532249873031996
 log loss: 0.38403349918046614


In [51]:
clf.best_params_

{'class_weight': 'balanced',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 100}

## Prediccion usando Random Forest

In [52]:
y_pred_probs =clf.predict_proba( df_test.drop(['Sales_Contract_No','Stage'], axis=1)  )[:,1]

In [53]:
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
prediccion = df_test['Opportunity_ID'].to_frame()
prediccion.insert(1,'Target', y_pred_probs , True) 
#Exportamos la prediccion en formato csv sin labels
prediccion.to_csv(r'./RandomForest_max_depth_none_class_weight_none_min_samples_leaf_1_min_samples_split_2_n_estimators_100_max_features_log2.csv',index=False , header=True)
prediccion.head()

,Opportunity_ID,Target
0,12173,0.467776
1,12172,0.433618
2,11408,0.568561
3,12232,0.446158
4,12233,0.478548


# Ensamble de arboles

In [127]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier

In [128]:
decisionTree = tree.DecisionTreeClassifier(class_weight=None,max_depth=5,min_samples_leaf=1,min_samples_split=3)

In [129]:
randomForest = RandomForestClassifier(class_weight='balanced',max_depth=None,min_samples_leaf=2,min_samples_split=2,n_estimators=100)

In [130]:
#logisticRegression = LogisticRegression(class_weight=None, max_iter=50)

In [131]:
#multiLayerPerceptron = MLPClassifier(random_state=1,activation='tanh', hidden_layer_sizes=(200, 200, 200, 200))


In [132]:
#kNeighbors =KNeighborsClassifier(n_neighbors=997)

In [133]:
XGBoost = XGBClassifier(n_estimators=50)

In [134]:
eclf = VotingClassifier(estimators=[('XGBoost',XGBoost),('dt', decisionTree), ('rf',randomForest)], voting='soft')

In [135]:
eclf = eclf.fit(X_train, y_train)

C:\Users\licha\Miniconda3\envs\orgadatos\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:18:31] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [136]:
y_validation =eclf.predict(X_test)
y_validation_probs = eclf.predict_proba(X_test)[:,1]

In [137]:
print( "Resultado:")
print( " accurancy:" , accuracy_score(y_test, y_validation) )
print( " log loss:" , log_loss(y_test, y_validation_probs) )

Resultado:
 accurancy: 0.8486541391569324
 log loss: 0.36119934711432194


## Prediccion usando ensamble de arboles

In [138]:
y_pred_probs =eclf.predict_proba( df_test.drop(['Sales_Contract_No','Stage'], axis=1)  )[:,1]

In [139]:
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
prediccion = df_test['Opportunity_ID'].to_frame()
prediccion.insert(1,'Target', y_pred_probs , True) 
#Exportamos la prediccion en formato csv sin labels
prediccion.to_csv(r'./EnsambleDeRandomForest_XGBoost_DesicionTree.csv',index=False , header=True)
prediccion.head()

,Opportunity_ID,Target
0,12173,0.236615
1,12172,0.384900
2,11408,0.748496
3,12232,0.702126
4,12233,0.760842
